In [1]:
import os

datapath = os.getcwd()+'\\data'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [9]:
TrainData = pd.read_csv(datapath + './train.csv')
ValData = pd.read_csv(datapath + './val.csv')
TestData = pd.read_csv(datapath + './test.csv')

In [16]:
val_normal, val_fraud = ValData['Class'].value_counts()
val_contamination = val_fraud / val_normal

print('prob :',val_contamination)

prob : 0.0010551491277433877


In [17]:
train_x = TrainData.drop(columns=['ID'])

In [18]:
model = EllipticEnvelope(support_fraction = 0.994, contamination = val_contamination, random_state = 42)
model.fit(train_x)

EllipticEnvelope(contamination=0.0010551491277433877, random_state=42,
                 support_fraction=0.994)

In [19]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [21]:
val_x = ValData.drop(columns=['ID', 'Class']) # Input Data
val_y = ValData['Class'] # Label

val_pred = model.predict(val_x) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))

tn, fn, fp, tp = confusion_matrix(val_y, val_pred).ravel()
print('tp : ', tp, ', fp : ', fp, ', tn : ', tn, ', fn : ', fn)

Validation F1 Score : [0.8927516353661109]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.85      0.73      0.79        30

    accuracy                           1.00     28462
   macro avg       0.92      0.87      0.89     28462
weighted avg       1.00      1.00      1.00     28462

tp :  22 , fp :  8 , tn :  28428 , fn :  4
